In [455]:
import random
import numpy as np
import pandas as pd
from __future__ import division
#Question 1
#Random motion 
class Location():
    
    def __init__(self, x, y):
        """x and y are floats"""
        self.x = x
        self.y = y
        
    def move(self, deltaX, deltaY):
        """deltaX and deltaY are floats"""
        return Location(self.x + deltaX, self.y + deltaY)
       
    def distFrom(self, other):
        xDist = self.x - other.x
        yDist = self.y - other.y
        return (xDist**2 + yDist**2)**0.5
    
    def __str__(self):
        return '<' + str(self.x) + ', ' + str(self.y) + '>'

class City():
    #Possible to add multiple tourists
    def __init__(self):
        self.tourists = {}
            
    def moveTourist(self, tourist):
        #Makes tourist take step and then update location
        xDist, yDist = tourist.takeStep()
        currentLocation = self.tourists[tourist]
        
        #Use Location's move method
        self.tourists[tourist] = currentLocation.move(xDist, yDist)
      

class Tourist():

    def takeStep(self):
        #The possible options are north, south, east and west.
        #(x, y) -> (0,1) is North, (1, 0) East, etc.
        stepChoices = [(0.0,1.0), (0.0,-1.0), (1.0, 0.0), (-1.0, 0.0)]
        return random.choice(stepChoices)



def walk(city, tourist, numSteps, travel_range=None):
    #Gets location of tourist in city
    start = city.tourists[tourist]
    max_distance=0
    east=0
    west=0
    east_west=0
    last_step=0
    
    for s in range(numSteps):
        #Moves tourist multiple times
        city.moveTourist(tourist)
        current_distance = start.distFrom(city.tourists[tourist])
        #Check for max distance
        if current_distance > max_distance:
            max_distance = current_distance
        #Check if present on E and W side
        if city.tourists[tourist].x > 1:
            east=1
        if city.tourists[tourist].x < -1:
            west=1
        if east and west:
            east_west=1
        
        #Check if travel_range reached
        if current_distance >= travel_range and travel_range is not None:
            last_step=s+1
            break
    return((current_distance, max_distance, east_west, last_step))



def simWalks(numSteps, numTrials, travel_range=None):
    
    bob = Tourist()
    origin = Location(0, 0)
    distances = []
    max_distances = []
    both_sides = []
    last_steps = []
    for t in range(numTrials):
        Grandeville = City()
        Grandeville.tourists[bob]= origin
        result = walk(Grandeville, bob, numSteps, travel_range)
        distances.append(result[0])
        max_distances.append(result[1])
        both_sides.append(result[2])
        last_steps.append(result[3])
    return (distances, max_distances, both_sides, last_steps)


sim_num=100
#odds of being more than 3 locations away after 10 steps
random.seed(0)
steps=10
data = pd.DataFrame(simWalks(steps, sim_num)[0], columns=['Distance'])
print('odds of being more than 3 locations away after 10 steps')
print('{0:.10f}'.format(sum(data.Distance>3)/float(data.shape[0])))

#odds of being more than 10 locations away after 60 steps
random.seed(0)
steps=60
data = pd.DataFrame(simWalks(steps, sim_num)[0], columns=['Distance'])
print('odds of being more than 10 locations away after 60 steps')
print('{0:.10f}'.format(sum(data.Distance>10)/float(data.shape[0])))

#odds of ever being more than 5 locations away after 10 steps
random.seed(0)
steps=10
data = simWalks(steps, sim_num)
data = pd.DataFrame({'Distance': data[0],'Max_Distance': data[1] })
print('Odds of ever being more than 5 locations away after 10 steps')
print('{0:.10f}'.format(sum(data.Max_Distance>5)/float(data.shape[0])))

#Odds of ever being more than 10 locations away after 60 steps
random.seed(0)
steps=60
data = simWalks(steps, sim_num)
data = pd.DataFrame({'Distance': data[0],'Max_Distance': data[1] })
print('Odds of ever being more than 10 locations away after 60 steps')
print('{0:.10f}'.format(sum(data.Max_Distance>10)/float(data.shape[0])))

#Odds of being more than one block east of orgin, and finishing more than one block west
random.seed(0)
steps=10
data = simWalks(steps, sim_num)
data = pd.DataFrame({'Distance': data[0],'Max_Distance': data[1],'Both': data[2] })
print('Odds of being more than one block east of orgin, and finishing more than one block west')
print('10 Steps')
print('{0:.10f}'.format(sum(data.Both)/float(data.shape[0])))

steps=60
data = simWalks(steps, sim_num)
data = pd.DataFrame({'Distance': data[0],'Max_Distance': data[1],'Both': data[2] })
print('60 Steps')
print('{0:.10f}'.format(sum(data.Both)/float(data.shape[0])))

#Number of steps required to go a desired distance
random.seed(0)
steps=10000000
travel_range = 10
data = simWalks(steps, sim_num, travel_range)
data = pd.DataFrame({'Distance': data[0],'Max_Distance': data[1],'Both': data[2], 'Last' : data[3] })
print('Steps to go 10')
print('{0:.10f}'.format(data.Last.mean()))

random.seed(0)
steps=10000000
travel_range = 60
data = simWalks(steps, sim_num, travel_range)
data = pd.DataFrame({'Distance': data[0],'Max_Distance': data[1],'Both': data[2], 'Last' : data[3] })
print('Steps to go 60')
print('{0:.10f}'.format(data.Last.mean()))

odds of being more than 3 locations away after 10 steps
0.5400000000
odds of being more than 10 locations away after 60 steps
0.2700000000
Odds of ever being more than 5 locations away after 10 steps
0.1400000000
Odds of ever being more than 10 locations away after 60 steps
0.3700000000
Odds of being more than one block east of orgin, and finishing more than one block west
10 Steps
0.0200000000
60 Steps
0.3600000000
Steps to go 10
104.9500000000
Steps to go 60
3802.0200000000


Question 2

In [461]:
import random
import numpy as np
import pandas as pd
from __future__ import division
data_sub = pd.read_csv('nyc311calls.csv.gz', usecols=['Agency', 'Agency Name', 'Borough','Latitude', 'Longitude', 'Complaint Type',  ])
agency=data_sub.groupby('Agency')
#Ratio of complaints to second largest agency
#(by complaint volume) divided by total number of complaints
answer = agency.size().sort_values(ascending =False)[1]/data_sub.shape[0]
print('Ratio of complaints to second largest agency')
print('{0:.10f}'.format(answer))

boroughs = data_sub.groupby(['Borough', 'Complaint Type'] )
complaints = boroughs.size()

#Odds of complaint type within borough
prob_complaint_B = complaints / complaints.sum(level=0)
#Odds of complaint type in general
prob_complaint = complaints/complaints.sum(level=1)

print("""the largest ratio of the conditional probability of a complaint type given a specified borough divided by the unconditioned probability of that complaint type""")
print((prob_complaint_B / prob_complaint).sort_values(ascending =False)[0])

print("""The distance (in degrees) between the 90% and 10% percentiles of degrees latitude""")
print('{0:.10f}'.format(data_sub.Latitude.quantile(.90)-data_sub.Latitude.quantile(.10)))

#Calculate area servered by 311 (How many square kilometers is the single-standard-deviation ellipse)
#Remove entries with missing gps data
x = data_sub.Latitude
x = x[x.isnull()==False]
y = data_sub.Longitude
y = y[y.isnull()==False]

#Find the ellipse angle, and then use Area = pi*stdx*stdy
dx2 = sum((x-np.mean(x))**2)
dy2 = sum((y-np.mean(y))**2)
A = dx2 - dy2
B = np.sqrt(A**2 + 4 * sum((x-np.mean(x))*(y-np.mean(y)))**2)
C = 2*sum((x-np.mean(x))*(y-np.mean(y)))
theta = np.arctan( (A+B)/C )
n = len(x)
Sx = np.sqrt(2* sum(((x-np.mean(x))*np.cos(theta) - (y-np.mean(y))*np.sin(theta))**2) / n)
Sy = np.sqrt(2* sum(((x-np.mean(x))*np.sin(theta) - (y-np.mean(y))*np.cos(theta))**2) / n)

#12365.1613 square km per degree
print('{0:.10f}'.format(np.pi*Sx*Sy*12365.1613) + ' square km')


#Find the difference between most calls in an hour and least in an hour
#Call Data
data_calls = pd.read_csv('nyc311calls.csv.gz', usecols=['Agency', 'Agency Name', 'Borough','Latitude', 'Longitude', 'Complaint Type', 'Created Date', 'Status' ])
data_calls['Created Date'] = pd.to_datetime(data_calls['Created Date'], format="%m/%d/%Y %I:%M:%S %p")

#An unlikely number of calls happened at 00:00:00,
#probably didn't have the time stamp data (3512710/10012260)
#Removing from data.

odd_data = ((data_calls['Created Date'].dt.second == 0) &
    (data_calls['Created Date'].dt.hour == 0) &
    (data_calls['Created Date'].dt.minute == 0))

clean_data = data_calls[ ~odd_data]

hour = clean_data.groupby(clean_data['Created Date'].dt.hour)
sorted_calls = hour.size().sort_values()
print('Call Number Difference')
print('{0:.10f}'.format(sorted_calls.iloc[-1]-sorted_calls.iloc[0]))

#Find standard devation of time between calls

time = clean_data['Created Date'].sort_values()
time = (time-time.shift(1)).iloc[1:-1]
print('standard deviation of time between calls')
print('{0:.10f}'.format(np.std(time.dt.total_seconds())))

Ratio of complaints to second largest agency
0.1719314121
the largest ratio of the conditional probability of a complaint type given a specified borough divided by the unconditioned probability of that complaint type
1.83649665535
The distance (in degrees) between the 90% and 10% percentiles of degrees latitude
0.2357908310
356.5738542531square km
Call Number Difference
500063.0000000000
standard deviation of time between calls
56.6804736870


In [ ]:
#Question 3  Project

Wearable sensors have become increasingly popular over the last few years with the success of smartphones, fitness trackers, and smart watches. Current generations of phones even contain special low power cores the constantly monitor movement. This allows them to act as pedometers for example with a negligible impact on battery life. Other use cases exist as well. For example, with android phones you can set your phone to stay unlocked until it detects that it hasn't moved for some time. The newest version on android even puts your phone in deep sleep to save power if the phone has not moved for a set period of time. This I feel is just the tip of the iceberg with it being possible to apply the data to different applications.

I propose that we apply this data to advertising. For example, I show that just by using accelerometer data if it possible to differentiate between distracted states such as standing while talking, and walking while talking vs. standing, walking, and working at the computer. I argue that a person is more likely to interact with the advertisement if there is less competing stimulus from other people. This would allow a company to sell ads at a higher price if the persons phone lists them as not being distracted. 

To show the feasibly I use sensor data from UCI that only contains the raw x,y,z accelerometer data.  The data is labeled into 7 categories from 15 subjects.  (Working at Computer, Standing Up, Walking and Going Up/Down stairs, Standing, Walking, Going Up/Down Stairs, Walking while Talking, Standing while Talking.)

Figure 1 ![Figure 1](https://raw.githubusercontent.com/damienrj/data_incubator/master/data.png) shows an example of the data where I calculate the magnitude of the acceleration vs. time.  The plot is color coded to indicate the labeled state.   I then preformed feature engineering to generate more features included low pass and high pass filtered versions of the data to highlight different behaviors.  I also used a rolling average to calculate the average acceleration within a one second window for the data.  The data was then split into training and testing sets.  Applying a random forest classifier with k-folds cross validation gave a score of 93% to the training data.  Using the test data, I also archived 93%.  Figure 2 ![Matrix](https://raw.githubusercontent.com/damienrj/data_incubator/master/matrix.png) is a confusion matrix generated showing that the model is predicting the different labels.

This work clearly shows it is possible to differentiate between states,  and this is only using one accelerometer. Most smartphones have many more available sensors that would make it possible to archive even higher accuracy. 




In [463]:
import random
import numpy as np
import pandas as pd
from __future__ import division
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from scipy import signal
from sklearn.metrics import confusion_matrix
import seaborn as sns
#Sampling frequency of the accelerometer: 52 Hz 
labels={1: 'Working at Computer', 2:'Standing Up, Walking, and Going Up/Down Stairs',
        3:'Standing', 4:'Walking', 5:'Going Up/Down Stairs', 6: 'Walking and Talking', 7:'Standing and Talking'}

# Buterworth low pass filter
N  = 2    # Filter order
Wn = .1 # Cutoff frequency of 2.6 hz
B, A = signal.butter(N, Wn)

#High pass
Wn = .9 # Cutoff frequency of 24 hz
Bh, Ah = signal.butter(N, Wn, 'high')

data=[]
for a in range(1,16):
    df = pd.read_csv('ActivityData/' + str(a) + '.csv' , header=None)
    df.drop(0, axis=1, inplace=True)
    df.columns = ['Accx', 'Accy', 'Accz', 'Label']
    df['ID'] = a
    df['Accx_filt'] = signal.filtfilt(B,A, df.Accx)
    df['Accy_filt'] = signal.filtfilt(B,A, df.Accy)
    df['Accz_filt'] = signal.filtfilt(B,A, df.Accz)
    df['Accx_filt_h'] = signal.filtfilt(Bh,Ah, df.Accx)
    df['Accy_filt_h'] = signal.filtfilt(Bh,Ah, df.Accy)
    df['Accz_filt_h'] = signal.filtfilt(Bh,Ah, df.Accz)
    df['Mean_x'] = pd.rolling_mean(df.Accx, 52)
    df['Mean_y'] = pd.rolling_mean(df.Accy, 52)
    df['Mean_z'] = pd.rolling_mean(df.Accz, 52)
    if len(data)==0:
        data = df
    else:
        data = pd.concat([data, df])
        
    
data = data[~(data.Label==0)]
data['Mag']= np.sqrt(data.Accx**2 + data.Accy**2 + data.Accz**2)

#Remove edges of data where there is no rolling averages (~50 rows per subject)
data=data[~data.Mean_x.isnull()]

features = ['Mag', 'Accx', 'Accy', 'Accz', 'ID', 'Accx_filt',
            'Accy_filt', 'Accz_filt', 'Accx_filt_h', 'Accy_filt_h',
            'Accz_filt_h', 'Mean_x', 'Mean_y', 'Mean_z']

#Split into training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(data[features], data['Label'])

#Simple K-Fold cross validation. 10 folds.
cv = KFold(len(features_train), n_folds=10, indices=False)

#iterate through the training and test cross validation segments and
#run the classifier on each one, aggregating the results into a list
results = []
for traincv, testcv in cv:
    rf.fit(features_train[traincv], labels_train[traincv])
    results.append(rf.score(features_train[testcv], labels_train[testcv]))
    
    
print('Mean score ' + str(np.mean(results)))

#Train model on whole training set
rf = RandomForestClassifier(n_estimators=20, n_jobs=-1, verbose=1, min_samples_leaf=3, oob_score=True)
rf.fit(features_train, labels_train)

print('Test score ' + str(rf.score(features_test,labels_test)))
#Generate confusion_matrix
con_matrix = confusion_matrix(labels_test, rf.predict(features_test))
cm_normalized = cm.astype('float') / cm.sum(axis=1)


%matplotlib 

plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues)

plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.colorbar()
plt.xticks([0,1,2,3,4,5,6], [1,2,3,4,5,6,7])
plt.yticks([0,1,2,3,4,5,6], [1,2,3,4,5,6,7])
plt.grid(False)
plt.savefig("matrix.png")

#Make plot for user 12
user = data[data.ID==12]
user.Mag-=np.min(user.Mag)
user.Mag /= np.max(user.Mag)
legend_text=[]

t = np.arange(0, len(user.Label))/52
sns.set_palette(sns.color_palette("Set2", 8))
for a in range(1,8):
    plt.plot(t[np.array(user.Label==a)], user.Mag[user.Label==a], linewidth = 1)
    legend_text.append(labels[a])

plt.legend(legend_text,'best')
plt.xlabel('Time(s)')
plt.ylabel('Acceleration Magnitude (normalized)')
plt.title('Subject 12 Data')
plt.xlim(xmax=max(t))
plt.savefig("data.png")

Using matplotlib backend: MacOSX
